<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Bárbara Aguayo
- Nombre de alumno 2: Camila Figueroa


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/bluemili/MDS7202-Labs`

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00


# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
# Carga de datos
from google.colab import files
uploaded = files.upload()

Saving sales.csv to sales.csv


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df

<ipython-input-5-c795eb3a5b6d>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696
...,...,...,...,...,...,...,...,...,...,...,...,...
7451,7555,2018-12-31,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,1.5lt,2.52,13760
7452,7556,2018-12-31,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,1.5lt,2.18,16309
7453,7557,2018-12-31,Patra,38.24444,21.73444,168034,shop_6,kinder-cola,can,330ml,0.85,24378
7454,7558,2018-12-31,Thessaloniki,40.64361,22.93086,354290,shop_4,adult-cola,plastic,1.5lt,2.17,20691


## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [ ]:
labels = df.loc[:, "quantity"]
features = df.drop(columns=["quantity", "id"])

In [ ]:
# Inserte su código acá
from sklearn.model_selection import train_test_split

X_train, X_valtest, y_train, y_valtest = train_test_split(
    features,
    labels,
    test_size=0.3,
    random_state=30,
    shuffle=False #, stratify=labels,
)

X_val, X_test, y_val, y_test = train_test_split(
    X_valtest,
    y_valtest,
    test_size=(1/3),
    random_state=30,
    shuffle=False #, stratify=labels,
)

In [ ]:
print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de validación:", len(X_val))
print("Tamaño del conjunto de prueba:", len(X_test))

Tamaño del conjunto de entrenamiento: 5219
Tamaño del conjunto de validación: 1491
Tamaño del conjunto de prueba: 746


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         7456 non-null   int64         
 1   date       7456 non-null   datetime64[ns]
 2   city       7456 non-null   object        
 3   lat        7456 non-null   float64       
 4   long       7456 non-null   float64       
 5   pop        7456 non-null   int64         
 6   shop       7456 non-null   object        
 7   brand      7456 non-null   object        
 8   container  7456 non-null   object        
 9   capacity   7456 non-null   object        
 10  price      7456 non-null   float64       
 11  quantity   7456 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 699.1+ KB


In [ ]:
#Implemente un FunctionTransformer para extraer el día, mes y año de la variable date. Guarde estas variables en el formato categorical de pandas.
from sklearn.preprocessing import FunctionTransformer

def date_func1(df):

  df['day'] = df['date'].dt.day.astype('category')
  df['month'] = df['date'].dt.month.astype('category')
  df['year'] = df['date'].dt.year.astype('category')

  #df['day'] = df['date'].apply(lambda x: x.day)
  #df['month'] = df['date'].apply(lambda x: x.month)
  #df['year'] = df['date'].apply(lambda x: x.year)

  return df

In [ ]:
date_func = FunctionTransformer(date_func1)

In [ ]:
#Implemente un ColumnTransformer para procesar de manera adecuada los datos numéricos y categóricos. Use OneHotEncoder para las variables categóricas.

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.dummy import DummyRegressor

numerical_columns = df.select_dtypes('float').columns
categorical_columns = df.select_dtypes(["object", "category"]).columns

column_transformer = ColumnTransformer([('StandardScaler', StandardScaler(), numerical_columns),
                                        ('OneHotEncoder', OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist'), categorical_columns),
                                        ])

In [ ]:
#Guarde los pasos anteriores en un Pipeline, dejando como último paso el regresor DummyRegressor para generar predicciones en base a promedios.

pipeline = Pipeline([
    ('func', date_func),
    ('col_tranformer', column_transformer),
    ('dummy', DummyRegressor())
])

In [ ]:
#Entrene el pipeline anterior y reporte la métrica mean_absolute_error sobre los datos de validación.

pipeline.fit(X_train, y_train)

Pipeline(steps=[('func',
                 FunctionTransformer(func=<function date_func1 at 0x7d71244c2320>)),
                ('col_tranformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  Index(['lat', 'long', 'price'], dtype='object')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  Index(['city', 'shop', 'brand', 'container', 'capacity'], dtype='object'))])),
                ('dummy', DummyRegressor())])

In [ ]:
pred = pipeline.predict(X_val)

In [ ]:
from sklearn.metrics import mean_absolute_error

mae_dum = mean_absolute_error(y_val, pred)
print(f'MAE Val: {mae_dum:.2f}')

MAE Val: 12499.05


**¿Cómo se interpreta esta métrica para el contexto del negocio?**

En este caso, el MAE se puede interpretar como cuánto, en promedio, el modelo se desvía de la cantidad real de productos vendidos.

In [ ]:
#Finalmente, vuelva a entrenar el Pipeline pero esta vez usando XGBRegressor como modelo utilizando los parámetros por default. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el DummyRegressor?

from xgboost import XGBRegressor

pipeline_xg = Pipeline([
    ('func', date_func),
    ('col_tranformer', column_transformer),
    ('xgb', XGBRegressor())
])


In [ ]:
pipeline_xg

Pipeline(steps=[('func',
                 FunctionTransformer(func=<function date_func1 at 0x7d71244c2320>)),
                ('col_tranformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  Index(['lat', 'long', 'price'], dtype='object')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  Index(['city', 'shop', 'brand', 'container', '...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
pipeline_xg.fit(X_train, y_train)

Pipeline(steps=[('func',
                 FunctionTransformer(func=<function date_func1 at 0x7d71244c2320>)),
                ('col_tranformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  Index(['lat', 'long', 'price'], dtype='object')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  Index(['city', 'shop', 'brand', 'container', '...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
pred_xg = pipeline_xg.predict(X_val)

In [ ]:
mae_xg = mean_absolute_error(y_val, pred_xg)
print(f'MAE Val: {mae_xg:.2f}')

MAE Val: 6338.03


**¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el DummyRegressor?**

Utilizando XGBoost se obtiene una gran mejora en el valor de MAE, dado que disminuye. Eso quiere decir que XGBRegressor es mejor que DummyRegressor para rpedecir la demanda de la cantidad.

In [ ]:
import pickle

# Guardar el mejor modelo en un archivo .pkl
with open('pipeline_dummy.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

# Guardar el mejor modelo en un archivo .pkl
with open('pipeline_xg1.pkl', 'wb') as file:
    pickle.dump(pipeline_xg, file)


## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




In [ ]:
df_train = pd.concat((X_train, y_train), axis = 1)

In [ ]:
df_train

,date,city,lat,long,pop,shop,brand,container,capacity,price,day,month,year,quantity
0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,31,1,2012,13280
1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,31,1,2012,6727
2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,31,1,2012,9848
3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,31,1,2012,20050
4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,31,1,2012,25696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5214,2016-11-30,Patra,38.24444,21.73444,168254,shop_6,lemon-boost,can,330ml,0.42,30,11,2016,40162
5215,2016-11-30,Patra,38.24444,21.73444,168254,shop_6,gazoza,plastic,1.5lt,0.77,30,11,2016,52204
5216,2016-11-30,Irakleion,35.32787,25.14341,137302,shop_2,orange-power,glass,500ml,1.00,30,11,2016,28926
5217,2016-11-30,Irakleion,35.32787,25.14341,137302,shop_2,gazoza,glass,500ml,0.62,30,11,2016,50738


In [ ]:
monotone_constraints = {"StandardScaler__price":-1}

#Se define el modelo con la restricción de monotonía
params = {
    'monotone_constraints': monotone_constraints
}

model = XGBRegressor(**params)

In [ ]:
pipeline_xg2 = Pipeline([
    ('func', date_func),
    ('col_tranformer', column_transformer),
    ('xgb', model)
])

In [ ]:
pipeline_xg2

Pipeline(steps=[('func',
                 FunctionTransformer(func=<function date_func1 at 0x7d71244c2320>)),
                ('col_tranformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  Index(['lat', 'long', 'price'], dtype='object')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  Index(['city', 'shop', 'brand', 'container', '...
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints={'StandardScaler__price': -1},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [ ]:
X_train_transformed = pipeline_xg2[0].fit_transform(X_train)

In [ ]:
X_train_transformed = pipeline_xg2[1].fit_transform(X_train_transformed)
X_train_transformed = pd.DataFrame(X_train_transformed, columns=pipeline_xg2[1].get_feature_names_out())

In [ ]:
X_train_transformed.columns

Index(['StandardScaler__lat', 'StandardScaler__long', 'StandardScaler__price',
       'OneHotEncoder__city_Athens', 'OneHotEncoder__city_Irakleion',
       'OneHotEncoder__city_Larisa', 'OneHotEncoder__city_Patra',
       'OneHotEncoder__city_Thessaloniki', 'OneHotEncoder__shop_shop_1',
       'OneHotEncoder__shop_shop_2', 'OneHotEncoder__shop_shop_3',
       'OneHotEncoder__shop_shop_4', 'OneHotEncoder__shop_shop_5',
       'OneHotEncoder__shop_shop_6', 'OneHotEncoder__brand_adult-cola',
       'OneHotEncoder__brand_gazoza', 'OneHotEncoder__brand_kinder-cola',
       'OneHotEncoder__brand_lemon-boost', 'OneHotEncoder__brand_orange-power',
       'OneHotEncoder__container_can', 'OneHotEncoder__container_glass',
       'OneHotEncoder__container_plastic', 'OneHotEncoder__capacity_1.5lt',
       'OneHotEncoder__capacity_330ml', 'OneHotEncoder__capacity_500ml'],
      dtype='object')

In [ ]:
pipeline_xg2[2].fit(X_train_transformed, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan,
             monotone_constraints={'StandardScaler__price': -1},
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
pred_xg2 = pipeline_xg2.predict(X_val)

In [ ]:
mae_xg2 = mean_absolute_error(y_val, pred_xg2)
print(f'MAE Val: {mae_xg2:.2f}')

MAE Val: 6157.60


**¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?**

Se observa una disminución en el valor del MAE, esto podría indicar que el amigo si tenía raón. Sin embargo, la disminución no es de gran magnitud por lo que esta relación entre la variable 'price' y la cantidad podría no ser tan relevante para la predicción final como para realizar una diferencia significativa.

## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [ ]:
# Inserte su código acá

In [ ]:
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
def objective_function(trial):
    # Split into features and target
    #le = LabelEncoder()
    #y = le.fit_transform(df.loc[:, "genre"])
    #X = df.drop(columns=["genre", "id", "name", "artist"]).astype(float)
    labels = df.loc[:, "quantity"]
    features = df.drop(columns=["quantity", "id"])

    X_train, X_valtest, y_train, y_valtest = train_test_split(
    features,
    labels,
    test_size=0.3,
    random_state=42,
    shuffle=False #, stratify=labels,
    )

    X_valid, X_test, y_valid, y_test = train_test_split(
        X_valtest,
        y_valtest,
        test_size=(1/3),
        random_state=42,
        shuffle=False #, stratify=labels,
    )


    #Los parámetros generales que se quieren optimizar
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }

    ohe_params = {
        "min_frequency": trial.suggest_float("min_frequency", 0.0, 1.0)
    }
    categorical_cols = ['city', 'shop', 'brand', 'container', 'capacity']
    numeric_cols = ['lat', 'long', 'pop', 'price']
    categorical_transformer = OneHotEncoder(min_frequency=ohe_params["min_frequency"], handle_unknown='ignore')
    numeric_transformer = StandardScaler()
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_cols),
            ('num', numeric_transformer, numeric_cols)
        ])

    X_train_processed = preprocessor.fit_transform(X_train)
    X_valid_processed = preprocessor.transform(X_valid)

    model = XGBRegressor(**params, objective='reg:squarederror')
    model.fit(X_train_processed, y_train,
              eval_set=[(X_valid_processed, y_valid)],
              early_stopping_rounds=10,
              verbose=False)


    yhat = model.predict(X_valid_processed)
    mae = mean_absolute_error(y_val, yhat)

    return mae

**Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?**

1. **learning_rate (0.001, 0.1)**: Este hiperpáremtro corresponde a los saltos de aprendizaje del modelo.
2. **n_estimators (50, 1000)**: Corresponde al número de árboles que se utilizarán en el modelo.
3. **max_depth (3, 10)**: Corresponde a la profundidad máxima de cada árbol.
4. **max_leaves (0, 100)**: Corresponde al número máximo de hojas en los árboles.
5. **min_child_weight (1, 5)**: Este hipérparámetro se refiere al peso mínimo que una hoja debe tener, permite prevenir la creación de hojas con pocos datos, lo cual ayuda a evitar el sobreajuste.
6. **reg_alpha (0, 1)**: Este hiperpárametro es el termino de regularización L1, ayuda a reducir el sobreajuste al añadir penalizaciones al tamaño de los coeficientes.
7. **reg_lambda (0, 1)**: Corresponde al termino de regularización L2. Similar a reg_alpha, pero añade una penalización cuadrática a los coeficientes.

  Para estos hiperparámetros se considera que los **rangos son adecuados**, dado que en general si los valores son más bajos o más altos que los expuestos pueden tender a causar subajuste o sobreajuste.

8. **min_frequency (0.0, 1.0)**: Corresponde a la frecuencia mínima que una categoría debe tener para ser codificada como una columna separada. Las categorías con frecuencias menores a este valor serán combinadas en una sola categoría "otros". Se considera que el rango es apropiado ya que permite flexibilidad en cómo se maneja la cardinalidad de las variables categóricas. Un rango de 0.0 a 1.0 cubre todos los casos posibles.

In [ ]:
from optuna.samplers import TPESampler
study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=42))

#Se define timeout=300 (300 segundos son 5 minutos)
study.optimize(objective_function, n_trials=100, timeout=300)

[I 2024-06-03 03:50:53,607] A new study created in memory with name: no-name-48a4ae96-373f-4fa7-a834-7bb8ac6f913d
<ipython-input-36-07cbbe4aa8b5>:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2024-06-03 03:50:56,458] Trial 0 finished with value: 8311.31020105005 and parameters: {'learning_rate': 0.005611516415334507, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352

In [ ]:
#Se reportan el número de trials, el MAE y los mejores hiperparámetros encontrados
print(f'Número de trials: {len(study.trials)}')
print(f'Mejor MAE: {study.best_value}')
print('Mejores hiperparámetros: ')
for key, value in study.best_params.items():
    print(f'{key}: {value}')

Número de trials: 100
Mejor MAE: 5983.112217903777
Mejores hiperparámetros: 
learning_rate: 0.09941898202295199
n_estimators: 423
max_depth: 4
max_leaves: 64
min_child_weight: 3
reg_alpha: 0.49842271204773153
reg_lambda: 0.47991057357608985
min_frequency: 0.12778498779733913


**¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?**

In [ ]:
#Se entrena el mejor modelo con los mejores hiperparámetros
best_params = study.best_params
best_params['objective'] = 'reg:squarederror'
#best_params['monotone_constraints'] = monotone_constraints
best_model = XGBRegressor(**best_params)

#Se crea el mejor pipeline
best_column_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(min_frequency=best_params['min_frequency']), categorical_columns)
    ])

best_pipeline = Pipeline([
    ('func', date_func),
    ('col_transformer', best_column_transformer),
    ('xgb', best_model)
])

#Se entrena el mejor pipeline
best_pipeline.fit(X_train, y_train)

#Se guarda el mejor modelo en un archivo .pkl
# Guardar el mejor modelo en un archivo .pkl
with open('pipeline_xg_optuna.pkl', 'wb') as file:
    pickle.dump(best_pipeline, file)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:53:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_frequency" } are not used.

  warnings.warn(smsg, UserWarning)


## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

**¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?**

El prunning es una técnica que permite descartar de forma rápida caminos de modelos que son poco prometedores, es decir, que se identifiquen los modelos que no lograrán un buen rendimiento. Esto debería impactar en el tiempo, reduciendolo y haciendo más eficiente el entrenamiento.   

In [ ]:
# Inserte su código acá
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
def objective_function(trial):
    # Split into features and target
    #le = LabelEncoder()
    #y = le.fit_transform(df.loc[:, "genre"])
    #X = df.drop(columns=["genre", "id", "name", "artist"]).astype(float)
    labels = df.loc[:, "quantity"]
    features = df.drop(columns=["quantity", "id"])

    X_train, X_valtest, y_train, y_valtest = train_test_split(
    features,
    labels,
    test_size=0.3,
    random_state=42,
    shuffle=False #, stratify=labels,
    )

    X_valid, X_test, y_valid, y_test = train_test_split(
        X_valtest,
        y_valtest,
        test_size=(1/3),
        random_state=42,
        shuffle=False #, stratify=labels,
    )


    #Los parámetros generales que se quieren optimizar
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }

    # Train the XGBoost model with prunning
    pruning_callback = optuna.integration.XGBoostPruningCallback(
        trial, observation_key="validation_1-mlogloss"
    )

    #Parámetro de OneHotEncoder a optimizar
    ohe_params = {
        "min_frequency": trial.suggest_float("min_frequency", 0.0, 1.0)
    }
    categorical_cols = ['city', 'shop', 'brand', 'container', 'capacity']
    numeric_cols = ['lat', 'long', 'pop', 'price']
    categorical_transformer = OneHotEncoder(min_frequency=ohe_params["min_frequency"], handle_unknown='ignore')
    numeric_transformer = StandardScaler()
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_cols),
            ('num', numeric_transformer, numeric_cols)
        ])

    X_train_processed = preprocessor.fit_transform(X_train)
    X_valid_processed = preprocessor.transform(X_valid)

    model = XGBRegressor(**params, objective='reg:squarederror')
    model.fit(X_train_processed, y_train,
              eval_set=[(X_valid_processed, y_valid)],
              early_stopping_rounds=10,
              verbose=False)


    yhat = model.predict(X_valid_processed)
    mae = mean_absolute_error(y_val, yhat)

    return mae

In [ ]:
from optuna.samplers import TPESampler
study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=42))
optuna.logging.set_verbosity(optuna.logging.WARNING)
#Se define timeout=300 (300 segundos son 5 minutos)
study.optimize(objective_function, n_trials=100, timeout=300, show_progress_bar = True)

[I 2024-06-03 03:53:45,796] A new study created in memory with name: no-name-e2171395-be3e-4b62-8cac-d5915d48360a


  0%|          | 0/100 [00:00<?, ?it/s]

[W 2024-06-03 03:53:45,895] Trial 0 failed with parameters: {} because of the following error: KeyError('quantity').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py", line 3653, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'quantity'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-44-134cae733865>", lin

KeyError: 'quantity'

In [ ]:
#Se reportan el número de trials, el MAE y los mejores hiperparámetros encontrados
print(f'Número de trials: {len(study.trials)}')
print(f'Mejor MAE: {study.best_value}')
print('Mejores hiperparámetros: ')
for key, value in study.best_params.items():
    print(f'{key}: {value}')

Número de trials: 100
Mejor MAE: 5983.112217903777
Mejores hiperparámetros: 
learning_rate: 0.09941898202295199
n_estimators: 423
max_depth: 4
max_leaves: 64
min_child_weight: 3
reg_alpha: 0.49842271204773153
reg_lambda: 0.47991057357608985
min_frequency: 0.12778498779733913


**¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?**

En este caso se obtuvieron los mismos resultados que la sección anterior. Esto puede deberse a que el proceso de Prunning no debiera cambiar realmente los valores encontrados de hiperparámetros, sino que debiera variar únicamente en el tiempo, realizando en menos segundos la optimización, dado que se descartaría de forma más rápida aquellos caminos que no son prometedores.

In [ ]:
#Se entrena el mejor modelo con los mejores hiperparámetros
best_params = study.best_params
best_params['objective'] = 'reg:squarederror'
#best_params['monotone_constraints'] = monotone_constraints
best_model = XGBRegressor(**best_params)

#Se crea el mejor pipeline
best_column_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(min_frequency=best_params['min_frequency']), categorical_columns)
    ])

best_pipeline_prun = Pipeline([
    ('func', date_func),
    ('col_transformer', best_column_transformer),
    ('xgb', best_model)
])

#Se entrena el mejor pipeline
best_pipeline_prun.fit(X_train, y_train)

#Se guarda el mejor modelo en un archivo .pkl
with open('pipeline_xg_prun.pkl', 'wb') as file:
    pickle.dump(best_pipeline_prun, file)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[02:41:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_frequency" } are not used.




## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
# Inserte su código acá

#Gráfico de historial de optimización
plot_optimization_history(study)

**¿Desde qué trial se empiezan a observar mejoras notables en sus resultados?**

Si bien se observa una mejora significativa en los primeros trials, se podría mencionar que a partir del trial **15** se muestra una tendencia al menor valor de la función objetivo.

In [ ]:
#Gráfico de coordenadas paralelas
plot_parallel_coordinate(study)

**¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?**

En primer lugar, se puede observar que los valores de "learning_rate" asociados con un menor valor en la función objetivo tienden a ser mayores dentro del rango especificado. Este comportamiento también se observa para el parámetro "reg_alpha". Por el contrario, los valores de "min_frequency" tienden a concentrarse en la parte inferior de su rango, al igual que los de "reg_lambda". En cuanto a "n_estimators", los valores observados tienden a concentrarse entre 300 y 500. Por otro lado, los otros hiperparámetros graficados muestran una mayor variabilidad en sus valores.

In [ ]:
#Gráfico de importancia de hiperparámetros
plot_param_importances(study)

**¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?**

De acuerdo al gráfico, se observa que el hiperparámetro con mayor importancia por lejos es min_frequency, seguido de la cantidad de estimadores de xgboost.

## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [ ]:
import pandas as pd

In [ ]:
m_optuna = 5983.11
m_pruning = 5983.11

data = {
    'Modelo': ['Baseline', 'XGBoost', 'XGBoost con Constraints', 'Optuna', 'Pruning'],
    'MAE': [mae_dum , mae_xg, mae_xg2, m_optuna, m_pruning]
}

df_results = pd.DataFrame(data)
df_results

,Modelo,MAE
0,Baseline,12499.045232
1,XGBoost,6338.032490
2,XGBoost con Constraints,6157.598525
3,Optuna,5983.110000
4,Pruning,5983.110000


Como se puede ver en la tabla, los resultados muestras que el peor modelo es el baseline, dado un MAE que es practicamente el doble de los que se obtuvieron con otros modelos. Tambien se puede observar que los mejores modelos corresponden al de Optuna y Pruning, los cuales alcanzan el mismo valor de MAE

In [ ]:
# Inserte su código acá
with open('pipeline_xg_prun.pkl', 'rb') as file:
    loaded_pipeline = pickle.load(file)

In [ ]:
pred_final = loaded_pipeline.predict(X_test)

In [ ]:
mae_final = mean_absolute_error(y_test, pred_final)
print(f'MAE Test: {mae_final:.2f}')

MAE Test: 7347.53


**¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?**

En este caso se observa un valor de MAE mayor para el conjunto de test en comparación al de validación. Esto podría deberse a que los datos de test se diferencian más que los de validación a los datos de entrenamiento.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>